# 02 – Model Development
### Student Depression Prediction: Building and Evaluating the XGBoost Classifier

---

## 📌 Objective  
Develop an end-to-end XGBoost classification pipeline to predict student depression using the preprocessed dataset.

**Key Tasks:**

 **Model Pipeline**  
- Train-test split  
- Categorical encoding
- Feature scaling/selection  

 **XGBoost Implementation**  
- Baseline model  
- Hyperparameter tuning  
- Cross-validation  

 **Model Evaluation**  
- Performance metrics (accuracy, precision, recall, F1, ROC-AUC)  
- Feature importance analysis  

---

### 📂 Input  
 - `clean_data.csv` saved in `Data/processed/FC110552_mithula-cbw/` (preprocessed dataset)


### 📦 Output  
- `xgb_model.pkl` saved in `models/FC110552_mithula-cbw/`

---

### 📈 Expected Outcomes
- Trained and validated XGBoost classification model for student depression prediction
- Performance evaluation metrics (accuracy, precision, recall, F1-score, ROC-AUC)
- Insights from feature importance analysis
- Model artifacts saved for future inference and deployment


## Step 1: Import Libraries & Load Data

In [93]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from prettytable import PrettyTable

# Scikit-learn modules for model selection and evaluation
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)

# XGBoost classifier
import xgboost as xgb


### Load preprocessed data

In [94]:
# Load preprocessed data
df = pd.read_csv('Data/processed/FC110552_mithula-cbw/clean_data.csv')

print("\n🔹 DataFrame Dimensions")
print("------------------------")
print(f"   Rows   : {df.shape[0]}")
print(f"   Columns: {df.shape[1]}")


🔹 DataFrame Dimensions
------------------------
   Rows   : 27868
   Columns: 13


In [95]:
# Print the first few rows of the data set
print(f"\n🔹 First 5 rows:")
df.head()


🔹 First 5 rows:


,gender,age,academic_pressure,cgpa,study_satisfaction,dietary_habits,have_you_ever_had_suicidal_thoughts,work_study_hours,financial_stress,family_history_of_mental_illness,depression,degree_encoded,sleep_duration_encoded
0,Male,33.0,5.0,8.97,2.0,Healthy,Yes,3.0,1.0,No,1,2,1
1,Female,24.0,2.0,5.90,5.0,Moderate,No,3.0,2.0,Yes,0,2,1
2,Male,31.0,3.0,7.03,5.0,Healthy,No,9.0,1.0,Yes,0,2,0
3,Female,28.0,3.0,5.59,2.0,Moderate,Yes,4.0,5.0,Yes,1,2,2
4,Female,25.0,4.0,8.13,3.0,Moderate,Yes,1.0,1.0,No,0,3,1


In [96]:
# Print data structure and datatypes of each column
table = PrettyTable()
table.field_names = ["Column", "Non-Null Count", "Dtype"]

for col in df.columns:
    non_null_count = df[col].count()
    dtype = df[col].dtype
    table.add_row([col, non_null_count, dtype])

print("\n🔹 Dataset Summary:")
print(table)


🔹 Dataset Summary:
+-------------------------------------+----------------+---------+
|                Column               | Non-Null Count |  Dtype  |
+-------------------------------------+----------------+---------+
|                gender               |     27868      |  object |
|                 age                 |     27868      | float64 |
|          academic_pressure          |     27868      | float64 |
|                 cgpa                |     27868      | float64 |
|          study_satisfaction         |     27868      | float64 |
|            dietary_habits           |     27868      |  object |
| have_you_ever_had_suicidal_thoughts |     27868      |  object |
|           work_study_hours          |     27868      | float64 |
|           financial_stress          |     27868      | float64 |
|   family_history_of_mental_illness  |     27868      |  object |
|              depression             |     27868      |  int64  |
|            degree_encoded           |   

💡 **Observations:**  
- No `id` column – No need to drop any identifier column.
- There are four object columns – These need encoding after the train-test split.

## Step 2: Train-Test Split

In [97]:
# Separate features and target
X = df.drop(['depression'], axis=1)  # Remove target column
y = df['depression']

> 🧠 To split the dataframe, we use stratified sampling to maintain the same  
class proportions in both the training and test sets.

> 🧠 Let's use an 80-20 split, where 80% of data is for training and 20% for testing.


In [98]:
# Stratified split to maintain class balance (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Print sizes of train and test sets
print(f"Training set size: {X_train.shape}")
print(f"Test set size:     {X_test.shape}")

# print depression rates in each split to verify stratification
print(f"Training set depression rate: {y_train.mean():.3f}")
print(f"Test set depression rate:     {y_test.mean():.3f}")

Training set size: (22294, 12)
Test set size:     (5574, 12)
Training set depression rate: 0.586
Test set depression rate:     0.586


💡 **Observations:**  
- The depression rate is consistent (58.6%) in both training and test sets.

> *🧠 confirming successful stratification and preserving class distribution for unbiased model evaluation.*

## Step 3: Categorical Encoding

> *🧠 Categorical columns (`gender`, `dietary_habits`, `suicidal_thoughts`, `family_history`) should now be encoded since the data has been split.*

In [99]:
# Let's Identify categorical columns to encode
categorical_columns = [
    'gender', 
    'dietary_habits', 
    'family_history_of_mental_illness', 
    'have_you_ever_had_suicidal_thoughts'
]

In [100]:
# Print unique values in each categorical column to inspect categories before encoding
for col in categorical_columns:
    print(f"🔹 Unique values in {col}: {df[col].unique()}")


🔹 Unique values in gender: ['Male' 'Female']
🔹 Unique values in dietary_habits: ['Healthy' 'Moderate' 'Unhealthy']
🔹 Unique values in family_history_of_mental_illness: ['No' 'Yes']
🔹 Unique values in have_you_ever_had_suicidal_thoughts: ['Yes' 'No']


💡 **Observations:**  
- All categorical columns have a small, well-defined set of unique categories.

> *🧠 Since all categorical variables have limited distinct values,   
>label encoding is appropriate and straightforward.*

In [101]:
# Check for unseen categories before encoding
for col in categorical_columns:
    train_categories = set(X_train[col].unique())
    test_categories = set(X_test[col].unique())
    unseen = test_categories - train_categories
    if unseen:
        print(f"Warning: Unseen categories in test set for '{col}': {unseen}")


# Label encode categorical variables
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X_train[col + '_encoded'] = le.fit_transform(X_train[col])
    X_test[col + '_encoded'] = le.transform(X_test[col])
    label_encoders[col] = le

X_train.drop(columns=categorical_columns, inplace=True)
X_test.drop(columns=categorical_columns, inplace=True)


In [104]:
print("Train data preview after encoding:")
X_train.head(3)

Train data preview after encoding:


,age,academic_pressure,cgpa,study_satisfaction,work_study_hours,financial_stress,degree_encoded,sleep_duration_encoded,gender_encoded,dietary_habits_encoded,family_history_of_mental_illness_encoded,have_you_ever_had_suicidal_thoughts_encoded
20912,18.0,5.0,6.42,3.0,7.0,5.0,1,2,0,2,0,1
4158,33.0,1.0,6.08,2.0,10.0,2.0,2,1,1,2,1,1
26933,23.0,2.0,8.59,5.0,12.0,4.0,2,0,1,1,0,1


In [105]:
print("\nTest data preview after encoding:")
X_test.head(3)



Test data preview after encoding:


,age,academic_pressure,cgpa,study_satisfaction,work_study_hours,financial_stress,degree_encoded,sleep_duration_encoded,gender_encoded,dietary_habits_encoded,family_history_of_mental_illness_encoded,have_you_ever_had_suicidal_thoughts_encoded
16297,20.0,3.0,8.95,4.0,6.0,4.0,1,2,1,0,1,0
23609,21.0,5.0,7.75,2.0,10.0,5.0,2,2,0,2,1,1
13498,27.0,3.0,7.27,4.0,2.0,3.0,0,0,1,2,0,1


💡 **Observations:**  
- Original categorical columns dropped.
- Encoded columns are numeric and ready for modeling.